**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


preprocess(前処理) というタイトル、今までのは pre-preprocess

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
import numpy as np
import qgrid
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

ここまでの処理で生成したDataFrameをpickleで保存したのでそれを読む  
pickle にしておくと読み込みも一瞬。  
最初からcsvなどを読んで成形して・・・を行うコードを書かなくても良いので、一時的なセーブデータとしては重宝する！（日付のパースなどをやり直さなくて良いので高速）

In [ ]:
kafun = pd.read_pickle("kafun03.pkl")
kafun.head()

In [ ]:
kafun.tail()

**もしかして、NaNしかない行が多い？**

## 全部が NaN の行を消去する。  

[dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) を使う

デフォルトでは axisで設定した方向の中に、一つでもNaNがあると消去されてしまう。how="any"  
ドキュメントを見る。

In [ ]:
kafun = kafun.dropna(axis=0, how="all")
kafun.tail()

---


## データがNaNばかりの列を排除したい。
#### 各列の非NaNのデータ数を数えてplotする

In [ ]:
kafun.count().plot(figsize=(20,3));

### 非NaNのデータ数が 25000以下の列にさようならをする

どうやって絞るか？

In [ ]:
kafun.count()>25000

<br>

###  これで取れるデータは・・・・
<br>

**columns ( 列 ) の bool 配列**
<br>

Columns/Rowsの bool によるの選択には loc が使える！

In [ ]:
col_above_25k = kafun.count()>25000
kafun.loc[:, col_above_25k]

---

### copy() して、新たなdfを作成する。
#### copyをしないとコピー元データへの参照が残り・・・
#### SettingWithCopyWarning となったら、対症療法的に .copy() を試してみる（乱暴）

なぜcopyするのか。。。 SettingWithCopyWarning などで怒られる

In [ ]:
kafun = kafun.loc[:, col_above_25k].copy()
kafun.head() # すっきり

---

## NaNも0に変更する fillna()

#### 異常値の扱い方は色々あるだろうが、今回はとりあえず、NaNも異常値(負値)も0にする  


### [fillna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

In [ ]:
kafun = kafun.fillna(0)
kafun.head()

---


## 負の値を0に変更する

<br>

**忘れた頃に登場**

- -9998:降雪による欠測
- -9997:黄砂による欠測
- -9996:その他の欠測(前後の時間や周辺観測値と比較して不自然なデータ)
- 空白:未観測または通信障害による欠測

<br>
<br>

負の値を0に実現する方法は色々あるが、  
思いつきやすいのだと、 min/max系

もっと楽なやり方はないか・・・  

<br>
<br>

<br>

ドキュメントを探していると・・・  

### [clip()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.clip.html)

<br>



In [ ]:
kafun = kafun.clip(lower=0.0) # とてもらく
kafun.head()

### pickle にて保存

In [ ]:
kafun.to_pickle("kafun04.pkl")

## グラフをみる

In [ ]:
kafun.plot(figsize=(20,9), legend=True);

2013年の奈良強すぎでは？

<br>


#### [plot()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html) のオプションも調べる


In [ ]:
kafun.plot(figsize=(16,15), subplots=True);